In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#Score: 0.92533
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/audio-binary-classification/sample_submission.csv
/kaggle/input/audio-binary-classification/test_data.npy
/kaggle/input/audio-binary-classification/train_data.npy
/kaggle/input/audio-binary-classification/train_labels.csv


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import pandas as pd

Using TensorFlow backend.


In [3]:
# import mnist data and visualize first image
X_train = np.load("/kaggle/input/audio-binary-classification/train_data.npy")
X_test = np.load("/kaggle/input/audio-binary-classification/test_data.npy")
y_train = pd.read_csv('/kaggle/input/audio-binary-classification/train_labels.csv').loc[:,'Label'].values

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train = np.reshape(X_train, (-1, 210,210))
X_test = np.reshape(X_test, (X_test.shape[0], 210,210))


In [4]:
print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.20, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

model.summary()

Build LSTM RNN model ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 210, 64)           70400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 82,849
Trainable params: 82,849
Non-trainable params: 0
_________________________________________________________________


In [6]:

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 100, epochs = 50)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = model.predict(X_test)
#np.savetxt("y_pred_ann.csv", y_pred, delimiter=",")
  
df2 = pd.DataFrame(y_pred, index = range(y_pred.shape[0]))
df2.reset_index(level=0, inplace=True)
df2.columns=['Id', 'Label']
df2.to_csv("y_pred_LSTM_64_32.csv", index=False)


Epoch 1/50
3999/3999 [==============================] - 34s 9ms/step - loss: 0.4156 - acc: 0.8557
Epoch 2/50
3999/3999 [==============================] - 31s 8ms/step - loss: 0.2680 - acc: 0.8770
Epoch 3/50
3999/3999 [==============================] - 31s 8ms/step - loss: 0.2503 - acc: 0.8770
Epoch 4/50
3999/3999 [==============================] - 31s 8ms/step - loss: 0.2450 - acc: 0.8785
Epoch 5/50
3999/3999 [==============================] - 31s 8ms/step - loss: 0.2408 - acc: 0.8745
Epoch 6/50
3999/3999 [==============================] - 30s 8ms/step - loss: 0.2358 - acc: 0.8792
Epoch 7/50
3999/3999 [==============================] - 31s 8ms/step - loss: 0.2346 - acc: 0.8785
Epoch 8/50
3999/3999 [==============================] - 30s 8ms/step - loss: 0.2328 - acc: 0.8805
Epoch 9/50
3999/3999 [==============================] - 30s 8ms/step - loss: 0.2280 - acc: 0.8822
Epoch 10/50
3999/3999 [==============================] - 30s 8ms/step - loss: 0.2254 - acc: 0.8835
Epoch 11/50
3999/39